# Tuning models for proceedings classification

Choosing the best hyperparameters for CNN, Doc2Vec & TFIDF

## Setup

In [1]:
from numpy.random import seed
seed(9)
from tensorflow import set_random_seed
set_random_seed(9)

In [2]:
import random
import pandas as pd
import numpy as np
import re
import pickle
import unidecode
import time
import matplotlib.pyplot as plt
import seaborn as sns
import copy 
from random import choice
import time

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.similarities.index import AnnoyIndexer
from gensim.models.phrases import Phrases, Phraser
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

import keras
from itertools import product
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maiapolo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Main model (CNN)

## Preparing

Preparing dataset

In [3]:
with open("bases/mov.txt", "rb") as fp:   # Legal proceedings
    mov = pickle.load(fp)

with open("bases/tags.txt", "rb") as fp:   # Tags
    tags = pickle.load(fp)

Opening models that identify combinations of words as unique tokens:

In [4]:
bigrams=Phrases.load('modelos/bigrams_mov')
bibigrams=Phrases.load('modelos/bibigrams_mov')

Stopwords:

In [5]:
stop_words = set(stopwords.words('portuguese')) 

Letting motions in cronological order:

In [6]:
mov[0][:4]

[['14-11-2018',
  'Tipo do Movimento:Ato Ordinatório Praticado Descrição:Certifico e dou fé que as custas pendentes de fls 685 não foram recolhidas pelas parte GILMAR RODRIGUES REGO e BRUNO MEDEIROS.'],
 ['23-07-2018',
  'Tipo do Movimento:Juntada - Petição Descrição da juntada:Documento eletrônico juntado de forma automática.'],
 ['16-07-2018',
  'Tipo do Movimento:Publicado\xa0 Atos da Serventia Folhas do DJERJ.:388/416'],
 ['29-06-2018', 'Tipo do Movimento:Enviado para publicação ']]

In [7]:
for m in mov:
    m.reverse()

In [8]:
mov[0][:4]

[['10-09-2010',
  'Tipo do Movimento:Distribuição Sorteio Serventia:Cartório da 34ª Vara Cível - 34ª Vara Cível'],
 ['21-09-2010',
  'Tipo do Movimento:Conclusão ao Juiz Juiz:JOAO MARCOS DE CASTELLO BRANCO FANTINATO'],
 ['23-09-2010',
  'Tipo do Movimento:Despacho - Proferido despacho de mero expediente Descrição:Defiro JG. \r\n\r\nIndefiro a antecipação dos efeitos da tutela, eis que as alegações do autor carecem de dilação probatória.\r\n\r\nCite-se.'],
 ['23-09-2010', 'Tipo do Movimento:Enviado para publicação ']]

Defining functions we will use:

In [9]:
#Function that cleans texts:
def clean(resulta):   
    import copy 
   
    result = copy.deepcopy(resulta)
    
    result=result.lower()
    result=re.sub('\d', ' ', result)
    result=result.replace("lei ", "lei_")
    result=result.replace("lei nº ", "lei_")
    result=result.replace("lei n.º" ,"lei_")
    result=result.replace("lei estadual nº ", "lei_") 
    result=result.replace("lei federal nº ", "lei_") 
    result=result.replace("lei municipal nº ", "lei_")
    result=result.replace("fl. ", "fls. ")
    result=result.replace("fls. ", "fls_") 
    result=result.replace("p. ", "pp. ")
    result=result.replace("pp. ", "pp_")
    result=result.replace("art. ", "art_") 
    result=result.replace("artigo ", "art_")
    result=result.replace("inciso ", "inciso_") 
    result=result.replace("nº ", "nº_")
    result=result.replace("n° ", "nº_")
    result=result.replace("º ", "º")
    result=result.replace("ª ", "ª")
    result=result.replace("oab ", "oab_")
    result=result.replace("r$ ", "r$_")
    result=result.replace("\n", " ")
    result=result.replace("dr ", "dr_")
    result=result.replace("dr. ", "dr_")
    result=result.replace("dra ", "dr_")
    result=result.replace("dra. ", "dr_")
    result=result.replace("adv: ", "adv_") 
    
    result=result.replace("/", " ")
    result=result.replace("|", " ")
    result=result.replace("+", " ")
    result=result.replace(".", " ")
    result=result.replace(",", " ")
    result=result.replace(":", " ")
    result=result.replace(";", " ")
    result=result.replace("!", " ")
    result=result.replace("?", " ")
    result=result.replace(">", " ")
    result=result.replace("=", " ")
    result=result.replace("§", " ")
    result=result.replace(" - ", " ")
    result=result.replace(" _ ", " ")
    result=result.replace("&", " ")
    result=result.replace("*", " ")
    result=result.replace("(", " ")
    result=result.replace(")", " ")
    result=result.replace("ª", " ")
    result=result.replace("º", " ")
    result=result.replace("%", " ")
    result=result.replace("[", " ")
    result=result.replace("]", " ")
    result=result.replace("{", " ")
    result=result.replace("}", " ")
    result=result.replace("'", " ")
    result=result.replace('"', " ")
    result=result.replace("“", " ")
    result=result.replace("”", " ")
    result=re.sub(' +', ' ', result)

    return(result)

In [10]:
#Tokenizer
stop_words = set(stopwords.words('portuguese'))

def tokenize(txt):
    texto=unidecode.unidecode(txt) #decodificando
    texto=clean(texto) #limpando texto
    texto=texto.split(' ') 
    
    tokens=[]
    for t in texto:
        if t not in stop_words: tokens.append(t)
        else: pass
        
    tokens=bibigrams[bigrams[tokens]]
 
    return(tokens)

# CNN

Opening Word2Vec pre-trained model:

In [11]:
word = KeyedVectors.load("modelos/doc2vec_mov_100_5_V5")

embed_dim=np.shape(word['juiz'])[0]
embed_dim

100

Our vocabulary:

In [12]:
word_vectors = word.wv

Function for embedding (normalized):

In [13]:
def emb(s):
    return(word[s]/np.sqrt(word[s]@word[s]))

In [14]:
#Transform a sequence of tokens to the matrix form, 
#where each line is given by an embedded token
def tokens2matrix(tokens):
    matrix=[]
    for s in tokens: 
        if (s in word_vectors.vocab) and (s not in stop_words): #in vocab and no stopwords
            matrix.append(list(emb(s)))
        else: pass
    return(np.array(matrix))

In [15]:
#function that checks if in a given text there is at least one word in the vocabulary
def in_vocab(texto):
    for s in tokenize(texto):
        if (s in word_vectors.vocab) and (s not in stop_words): return(True)
        else: pass
    return(False)

In [16]:
def get_X_y_cnn(mov,tags,len1=5,len2=30):
    
    #len1: let's take the last len1 motions in each proceeding
    #len2: let's take the first len2 tokens in each motion
    
    #indexes of valid legal proceedings (with motions and tags)
    index=[]
    for i in range(len(mov)):
        if 0<len(mov[i]) and tags[i]!="": index.append(i) 
        else: pass

    print("Number of valid proceedings (with motions and tags):",len(index),"\n")

    #organizing motions in X and tags in y
    X=np.zeros((len(index),len1,len2,embed_dim))
    y=[]
    cont=0
    
    #get numerical X embedding the tokens from those motions (from valid proceedings):
    for i in index:
        temp=[]
        y.append(tags[i])

        mov[i]=mov[i][-len1::] #let's take the last len1 motions in each proceeding

        for j in range(len(mov[i])):

            if in_vocab(mov[i][j][1]):  #if there is at least one word in the vocabulary
                temp=tokens2matrix(tokenize(mov[i][j][1])[:len2]) #let's take the first len2 tokens in each motion and turn them into a matrix l2x100
                X[cont][j][:np.shape(temp)[0]]=temp
            else: pass


        #counter
        cont+=1
        if cont%int(len(index)/5)==0: print(round(100*cont/len(index),0),"% concluded")
        else: pass
        
    return X,y

In [17]:
len1=5 #vamos pegar somente as últimas l1 movimentações
len2=30 #vamos pegar somente os primeiros l2 tokens de cada movimentação

X,y=get_X_y_cnn(mov,tags,len1,len2)

Number of valid proceedings (with motions and tags): 6449 

20.0 % concluded
40.0 % concluded
60.0 % concluded
80.0 % concluded
100.0 % concluded


Turning y into numeric:

In [18]:
encode={'H:Arquivado': 1,'H:Ativo': 2,'H:Suspenso': 3}
decode={1:'H:Arquivado',2:'H:Ativo',3:'H:Suspenso'}
#
for i in range(len(y)):
    y[i]=encode[y[i]]

Splitting the dataset in train, test and validation set:

In [19]:
y=np.array(y)
#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=2/3, random_state=22)
#
y_train2=np.array(pd.get_dummies(y_train))
y_val2=np.array(pd.get_dummies(y_val))
y_test2=np.array(pd.get_dummies(y_test))

np.shape(X_train),np.shape(y_train)

((4514, 5, 30, 100), (4514,))

### Tuning classification model

Grid

In [87]:
def expand_grid(dictionary):
       return pd.DataFrame([row for row in product(*dictionary.values())], 
                           columns=dictionary.keys())

hyper = {'ks': [3,5,8,12], #kernels
         'neurons': [10,30,50,75,100], #hidden LSTM
         'lambdas': [.0,.0001,.0003,.0005,.0007,.0009,.0011,.0013, .0015, .0016, .0018, .002, .0025, .003], #regularization
         'score': [0], 
         'lower_ci': [0], 
         'upper_ci': [0]}

hyper=expand_grid(hyper)
hyper=hyper[['ks','neurons','lambdas','score','lower_ci','upper_ci']]

np.shape(hyper)

(280, 6)

Search

In [27]:
Adam=keras.optimizers.Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, amsgrad=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [138]:
start_time = time.time()

num_classes = 3

for i in range(np.shape(hyper)[0]):
    
    k=hyper.loc[i,'ks'] 
    neuron=hyper.loc[i,'neurons'] 
    lamb=hyper.loc[i,'lambdas'] 
    
    seed(42)
    set_random_seed(42)

    ### model for features extraction
    inputs = Input(shape=np.shape(X_train)[1:])
    conv = TimeDistributed(Conv1D(k, 1, activation='relu',kernel_constraint=unit_norm(axis=1), use_bias=False))(inputs)
    pool = TimeDistributed(GlobalMaxPooling1D())(conv)
    #
    model_feat = Model(inputs, pool)

    ### model for classification
    pooled_inputs = Input(shape=(5, k))
    lstm = LSTM(neuron, kernel_regularizer=l1(lamb))(pooled_inputs)
    soft = Dense(num_classes, activation='softmax')(lstm)
    #
    model_classific = Model(pooled_inputs, soft)

    ### final model
    outputs = model_classific(model_feat(inputs))
    model = Model(inputs, outputs)

    #compiling
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    #Running
    seed(42)
    set_random_seed(42)

    modelo=model.fit(X_train, y_train2, epochs=50,
                                              batch_size=500,
                                              shuffle=True,
                                              verbose=False,
                                              validation_data=(X_val, y_val2))
    
    p=modelo.history['val_accuracy'][-1]
    hyper.loc[i,'score']=p
    hyper.loc[i,'lower_ci']=p-1.96*np.sqrt((p*(1-p)/np.shape(y_val)[0]))
    hyper.loc[i,'upper_ci']=p+1.96*np.sqrt((p*(1-p)/np.shape(y_val)[0]))
    
    hyper.to_csv('hyper_cnn')
    
    #progress
    if i%int(np.shape(hyper)[0]/10)==0: print(round(100*i/np.shape(hyper)[0],0),"% concluded in", np.round((time.time() - start_time)/60,2),"minutes")
    else: pass

0.0 % concluded in 1.91 minutes
10.0 % concluded in 60.94 minutes
20.0 % concluded in 122.12 minutes
30.0 % concluded in 198.04 minutes
40.0 % concluded in 279.9 minutes
50.0 % concluded in 369.87 minutes
60.0 % concluded in 466.01 minutes
70.0 % concluded in 556.53 minutes
80.0 % concluded in 651.08 minutes
90.0 % concluded in 751.99 minutes


In [120]:
hyper=pd.read_csv('hyper_cnn')

In [122]:
hyper.iloc[np.argsort(hyper.loc[:,'score']),:].tail(50)

,Unnamed: 0,ks,neurons,lambdas,score,lower_ci,upper_ci
72,72,8,50,0.0005,0.894574,0.870873,0.918274
71,71,8,30,0.0005,0.894574,0.870873,0.918274
113,113,8,75,0.0009,0.894574,0.870873,0.918274
88,88,5,75,0.0007,0.894574,0.870873,0.918274
14,14,8,100,0.0000,0.894574,0.870873,0.918274
196,196,12,30,0.0016,0.896124,0.872578,0.919670
236,236,12,30,0.0020,0.896124,0.872578,0.919670
48,48,5,75,0.0003,0.896124,0.872578,0.919670
93,93,8,75,0.0007,0.896124,0.872578,0.919670
156,156,12,30,0.0013,0.897674,0.874285,0.921064


# Doc2Vec

In [50]:
doc = Doc2Vec.load("modelos/doc2vec_mov_100_5_V5")  

In [51]:
def emb_doc(tokens, model, normalize=True): #a opção normalize pode melhorar a qualidade do embedding
    
    model.random.seed(42)
    x=model.infer_vector(tokens, steps=20)
    
    if normalize: return(x/(np.sqrt(x@x)))
    else: return(x)

In [52]:
def get_X_y_doc(mov,tags,len1=5):
    
    #len1: let's take the last l1 motions in each proceeding
    
    #indexes of valid legal proceedings (with motions and tags)
    index=[]
    for i in range(len(mov)):
        if 0<len(mov[i]) and tags[i]!="": index.append(i) 
        else: pass

    print("Number of valid proceedings (with motions and tags):",len(index),"\n")

    #organizing motions in X and tags in y
    X=np.zeros((len(index),len1,embed_dim))
    y=[]
    cont=0
    
    #get numerical X embedding the tokens from those motions (from valid proceedings):
    for i in index:
        temp=[]
        y.append(tags[i])

        mov[i]=mov[i][-len1::] #let's take the last l1 motions in each proceeding

        for j in range(len(mov[i])):
            temp=emb_doc(tokenize(mov[i][j][1]), doc)
            X[cont][j][:np.shape(temp)[0]]=temp

        #counter
        cont+=1
        if cont%int(len(index)/5)==0: print(round(100*cont/len(index),0),"% concluded")
        else: pass
        
    return X,y

In [53]:
len1=5 #vamos pegar somente as últimas len1 movimentações

X,y=get_X_y_doc(mov,tags,len1)

Number of valid proceedings (with motions and tags): 6449 

20.0 % concluded
40.0 % concluded
60.0 % concluded
80.0 % concluded
100.0 % concluded


In [54]:
encode={'H:Arquivado': 1,'H:Ativo': 2,'H:Suspenso': 3}
decode={1:'H:Arquivado',2:'H:Ativo',3:'H:Suspenso'}
#
for i in range(len(y)):
    y[i]=encode[y[i]]

In [55]:
y=np.array(y)
#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=2/3, random_state=22)
#
y_train2=np.array(pd.get_dummies(y_train))
y_val2=np.array(pd.get_dummies(y_val))
y_test2=np.array(pd.get_dummies(y_test))

np.shape(X_train),np.shape(y_train)

((4514, 5, 100), (4514,))

In [ ]:
def expand_grid(dictionary):
       return pd.DataFrame([row for row in product(*dictionary.values())], 
                           columns=dictionary.keys())

hyper = {'neurons': [10,30,50,75,100], #hidden LSTM
         'lambdas': [.0,.0001,.0003,.0005,.0007,.0009,.0011,.0013, .0015, .0016, .0018, .002, .0025, .003], #regularization
         'score': [0], 
         'lower_ci': [0], 
         'upper_ci': [0]}

hyper=expand_grid(hyper)
hyper=hyper[['neurons','lambdas','score','lower_ci','upper_ci']]

np.shape(hyper)

In [ ]:
import time

In [120]:
Adam=keras.optimizers.Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [123]:
start_time = time.time()

num_classes = 3

for i in range(np.shape(hyper)[0]):
    
    neuron=hyper.loc[i,'neurons'] 
    lamb=hyper.loc[i,'lambdas'] 
    
    seed(42)
    set_random_seed(42)

    ### model for features extraction
    inputs = Input(shape=np.shape(X_train)[1:]) 
    time_layer = TimeDistributed(Lambda(lambda x: x))(inputs) #Dense(neuron,activation='relu', activity_regularizer=l1(lamb1))
    lstm = LSTM(neuron, kernel_regularizer=l1(lamb))(time_layer)
    soft = Dense(num_classes, activation='softmax')(lstm)

    ### final model
    model = Model(inputs, soft)

    #compiling
    model.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

    #Running
    seed(42)
    set_random_seed(42)

    modelo=model.fit(X_train, y_train2, epochs=50,
                                              batch_size=500,
                                              shuffle=True,
                                              verbose=False,
                                              validation_data=(X_val, y_val2))
    
    p=modelo.history['val_accuracy'][-1]
    hyper.loc[i,'score']=p
    hyper.loc[i,'lower_ci']=p-1.96*np.sqrt((p*(1-p)/np.shape(y_val)[0]))
    hyper.loc[i,'upper_ci']=p+1.96*np.sqrt((p*(1-p)/np.shape(y_val)[0]))
   
    #progress
    if i%int(np.shape(hyper)[0]/10)==0: print(round(100*i/np.shape(hyper)[0],0),"% concluded in", np.round((time.time() - start_time)/60,2),"minutes")
    else: pass

0.0 % concluded in 0.25 minutes
10.0 % concluded in 2.13 minutes
20.0 % concluded in 4.19 minutes
30.0 % concluded in 6.92 minutes
40.0 % concluded in 9.9 minutes
50.0 % concluded in 13.63 minutes
60.0 % concluded in 17.62 minutes
70.0 % concluded in 22.42 minutes
80.0 % concluded in 27.61 minutes
90.0 % concluded in 33.61 minutes


In [124]:
hyper.iloc[np.argsort(hyper.loc[:,'score']),:].tail(30)

,neurons,lambdas,score,lower_ci,upper_ci
62,100,0.0011,0.820155,0.790515,0.849795
21,30,0.0013,0.820155,0.790515,0.849795
9,10,0.0016,0.820155,0.790515,0.849795
20,30,0.0011,0.821705,0.792166,0.851245
31,50,0.0005,0.823256,0.793817,0.852694
33,50,0.0009,0.823256,0.793817,0.852694
11,10,0.0020,0.824806,0.795469,0.854143
19,30,0.0009,0.824806,0.795469,0.854143
6,10,0.0011,0.826357,0.797123,0.855591
56,100,0.0000,0.826357,0.797123,0.855591


# TFIDF

In [61]:
#Tokenizador de textos
stop_words = set(stopwords.words('portuguese'))

def tokenize_tfidf(txt):
    texto=txt
    texto=texto.split(' ') 
    texto=bibigrams[bigrams[texto]]
    tokens=[]
    for t in texto:
        if t not in stop_words: tokens.append(t)
        else: pass
 
    return(tokens)

In [62]:
tfidf = pickle.load(open('modelos/tfidf_mov_V1.sav', 'rb'))
tfidf_dim=2000

In [63]:
def get_X_y_tfidf(mov,tags,len1=5):
    
    #len1: let's take the last l1 motions in each proceeding

    #indexes of valid legal proceedings (with motions and tags)
    index=[]
    for i in range(len(mov)):
        if 0<len(mov[i]) and tags[i]!="": index.append(i) 
        else: pass

    print("Number of valid proceedings (with motions and tags):",len(index),"\n")

    #organizing motions in X and tags in y
    X=np.zeros((len(index),len1,tfidf_dim))
    y=[]
    cont=0
    
    #get numerical X embedding the tokens from those motions (from valid proceedings):
    for i in index:
        temp=[]
        y.append(tags[i])

        mov[i]=mov[i][-len1::] #let's take the last l1 motions in each proceeding

        for j in range(len(mov[i])):
            temp=tfidf.transform([mov[i][j][1]]).toarray()[0]
            X[cont][j][:np.shape(temp)[0]]=temp

        #counter
        cont+=1
        if cont%int(len(index)/5)==0: print(round(100*cont/len(index),0),"% concluded")
        else: pass
        
    return X,y

In [64]:
len1=5 #vamos pegar somente as últimas len1 movimentações

X,y=get_X_y_tfidf(mov,tags,len1)

Number of valid proceedings (with motions and tags): 6449 

20.0 % concluded
40.0 % concluded
60.0 % concluded
80.0 % concluded
100.0 % concluded


In [65]:
encode={'H:Arquivado': 1,'H:Ativo': 2,'H:Suspenso': 3}
decode={1:'H:Arquivado',2:'H:Ativo',3:'H:Suspenso'}
#
for i in range(len(y)):
    y[i]=encode[y[i]]

In [66]:
y=np.array(y)
#
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=2/3, random_state=22)
#
y_train2=np.array(pd.get_dummies(y_train))
y_val2=np.array(pd.get_dummies(y_val))
y_test2=np.array(pd.get_dummies(y_test))

np.shape(X_train),np.shape(y_train)

((4514, 5, 2000), (4514,))

In [67]:
def expand_grid(dictionary):
       return pd.DataFrame([row for row in product(*dictionary.values())], 
                           columns=dictionary.keys())

hyper = {'neurons': [10,30,50,75,100], #hidden LSTM
         'lambdas': [.0,.0001,.0003,.0005,.0007,.0009,.0011,.0013, .0015, .0016, .0018, .002, .0025, .003], #regularization
         'score': [0], 
         'lower_ci': [0], 
         'upper_ci': [0]}

hyper=expand_grid(hyper)
hyper=hyper[['neurons','lambdas','score','lower_ci','upper_ci']]

np.shape(hyper)

(70, 5)

In [34]:
hyper=pd.read_csv('hyper_tfidf')

In [36]:
Adam=keras.optimizers.Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, amsgrad=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
start_time = time.time()

num_classes = 3

for i in range(68,70):
    
    neuron=hyper.loc[i,'neurons'] 
    lamb=hyper.loc[i,'lambdas'] 
    
    seed(42)
    set_random_seed(42)

    ### model for features extraction
    inputs = Input(shape=np.shape(X_train)[1:]) 
    time_layer = TimeDistributed(Lambda(lambda x: x))(inputs) #Dense(neuron,activation='relu', activity_regularizer=l1(lamb1))
    lstm = LSTM(neuron, kernel_regularizer=l1(lamb))(time_layer)
    soft = Dense(num_classes, activation='softmax')(lstm)

    ### final model
    model = Model(inputs, soft)

    #compiling
    model.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

    #Running
    seed(42)
    set_random_seed(42)

    modelo=model.fit(X_train, y_train2, epochs=50,
                                              batch_size=500,
                                              shuffle=True,
                                              verbose=False,
                                              validation_data=(X_val, y_val2))
    
    p=modelo.history['val_accuracy'][-1]
    hyper.loc[i,'score']=p
    hyper.loc[i,'lower_ci']=p-1.96*np.sqrt((p*(1-p)/np.shape(y_val)[0]))
    hyper.loc[i,'upper_ci']=p+1.96*np.sqrt((p*(1-p)/np.shape(y_val)[0]))
    

    hyper.to_csv('hyper_tfidf')
    #progress
    if i%int(np.shape(hyper)[0]/10)==0: print(round(100*i/np.shape(hyper)[0],0),"% concluded in", np.round((time.time() - start_time)/60,2),"minutes")
    else: pass

In [40]:
hyper.iloc[np.argsort(hyper.loc[:,'score']),:].tail(30)

,Unnamed: 0,neurons,lambdas,score,lower_ci,upper_ci
19,19,30,0.0009,0.927132,0.907072,0.947191
33,33,50,0.0009,0.927132,0.907072,0.947191
35,35,50,0.0013,0.928682,0.908821,0.948544
7,7,10,0.0013,0.928682,0.908821,0.948544
39,39,50,0.0020,0.928682,0.908821,0.948544
27,27,30,0.0030,0.928682,0.908821,0.948544
62,62,100,0.0011,0.928682,0.908821,0.948544
6,6,10,0.0011,0.930233,0.910572,0.949893
4,4,10,0.0007,0.930233,0.910572,0.949893
64,64,100,0.0015,0.930233,0.910572,0.949893
